In [1]:
from torch import load as torch_load
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

from lstm_utils.model import BiLSTM, LSTM
from lstm_utils.train import train, test
from lstm_utils.utils import vectorize_text
from datasets import load_dataset

/home/cybertech2/tomekg/NLP_learning/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset('emotion')
ds

/home/cybertech2/tomekg/NLP_learning/.venv/lib/python3.8/site-packages/datasets/load.py:1461: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
ds.set_format('pandas')

In [ ]:
train_ds = vectorize_text(X=ds['train']['text'], y=ds['train']['label'], max_num_of_words=20)
val_ds = vectorize_text(X=ds['validation']['text'], y=ds['validation']['label'], max_num_of_words=20)
test_ds = vectorize_text(X=ds['test']['text'], y=ds['test']['label'], max_num_of_words=20)

In [ ]:
BATCH_SIZE = 32

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
HIDDEN_DIM = 128
NUM_LAYERS_LSTM = 2
DROPOUT_LSTM = 0.2
DROPOUT_CLF = 0.2
LEARNING_RATE = 0.01
NUM_EPOCHS = 100
PATIENCE = 20

lstm = LSTM(
    embedding_dim=20,
    hidden_dim=HIDDEN_DIM,
    output_dim=6,
    num_layers_lstm=NUM_LAYERS_LSTM,
    dropout_lstm=DROPOUT_LSTM,
    dropout_clf=DROPOUT_CLF
)
optimizer = Adam(lstm.parameters(), lr=LEARNING_RATE)

train(
    model=lstm,
    optimizer=optimizer,
    loss_fn=CrossEntropyLoss(),
    epochs=NUM_EPOCHS,
    train_dl=train_dl,
    val_dl=val_dl,
    patience=PATIENCE,
    print_metrics=True,
    device='cuda',
    best_model_path='lstm_utils/best_lstm.pth'
)

In [ ]:
lstm.load_state_dict(torch_load.load('lstm_utils/best_lstm.pth'))
lstm.eval()
test(
    model=lstm,
    dataloader=test_dl,
    device='cuda'
)

In [ ]:
HIDDEN_DIM = 128
NUM_LAYERS_LSTM = 2
DROPOUT_LSTM = 0.2
DROPOUT_CLF = 0.2
LEARNING_RATE = 0.01
NUM_EPOCHS = 100
PATIENCE = 20

bilstm = BiLSTM(
    embedding_dim=20,
    hidden_dim=HIDDEN_DIM,
    output_dim=6,
    num_layers_lstm=NUM_LAYERS_LSTM,
    dropout_lstm=DROPOUT_LSTM,
    dropout_clf=DROPOUT_CLF
)
optimizer = Adam(bilstm.parameters(), lr=LEARNING_RATE)

train(
    model=bilstm,
    optimizer=optimizer,
    loss_fn=CrossEntropyLoss(),
    epochs=NUM_EPOCHS,
    train_dl=train_dl,
    val_dl=val_dl,
    patience=PATIENCE,
    print_metrics=True,
    device='cuda',
    best_model_path='lstm_utils/best_bilstm.pth'
)

In [ ]:
bilstm.load_state_dict(torch_load.load('lstm_utils/best_bilstm.pth'))
bilstm.eval()
test(
    model=bilstm,
    dataloader=test_dl,
    device='cuda'
)